# Methods needed to summarize AWPY data

In [2]:
from awpy import DemoParser
from awpy.analytics.states import generate_game_state
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import glob


In [ ]:


# Set the parse_rate equal to the tick rate at which you would like to parse the frames of the demo.
# This parameter only matters if parse_frames=True ()
# For reference, MM demos are usually 64 ticks, and pro/FACEIT demos are usually 128 ticks.
def parse_folder(folderpath: str, parserate=64):
    
    ### initialize demo parser
    demo_parser = DemoParser(demofile="", demo_id="", parse_rate=parserate)

    ### gathering all demos in folder
    myfiles = [i for i in glob.glob(pathname=folderpath+"\*.dem")]

    ### parsing each demo in folder
    for n, file in enumerate(myfiles):
        demo_parser.demofile = file
        demo_parser.demo_id = "game"+str(n)
        data = demo_parser.parse()
        print(demo_parser.demofile, data["tickRate"])
        
# Input the path to the folder with your demos
parse_folder("Folder with demos")

In [ ]:

# initialize demo parser
demo_parser = DemoParser(demo_id="eg", parse_rate=64)

# create array of all rounds
allRounds = []

### get all jsons in folder and add all rounds of the game to allRounds
for i in glob.glob("*.json"):
    tempdata = demo_parser.read_json(i)
    for n, j in enumerate(tempdata["gameRounds"]):
        tempdata["gameRounds"][n]["tickRate"] = tempdata["tickRate"]
        tempdata["gameRounds"][n]["map"] = tempdata["mapName"]
    allRounds.extend(tempdata["gameRounds"])


### encoding winner within every frame
for ground in allRounds:
    for n, i in enumerate(ground["frames"]):
        i["winner"] = ground["winningSide"]
        i["seconds"] = (i["tick"] - ground["startTick"]) / ground["tickRate"]
        i["secondsLeft"] = (ground["endTick"] - i["tick"]) / ground["tickRate"]
        #print("Appending tick", i["tick"])
        ground["frames"][n] = i



In [11]:
# method for returning the sum of an attribute from each player
def sum_attribute(players: list, atrb: str):
    return sum(i[atrb] for i in players)

# returns the total number of the specified weapon in a player list / team
def sum_weapons(players: list, wpn: str):
    return sum([i["weaponName"]==wpn for j in [k["inventory"] if k["inventory"] != None else [] for k in players ] for i in j])

# prints weapons in the team
def print_weapons(players: list):
    return [i["weaponName"] for j in [k["inventory"] for k in players] for i in j]

# returns an array of dictionaries of the total number of each weapon for both teams
def return_team_inv(ctplayers: list, tplayers: list):
    wpsCode = ["ak47", "aug", "awp", "bizon", "cz75auto", "elite", "famas", "g3sg1", "galilar", "glock", "m249", "m4a1s", "m4a4", "mac10", "mag7", "mp5sd", "mp7", "mp9", "negev", "nova", "p90", "r8revolver", "sawedoff", "scar20", "sg553", "ssg08", "ump45", "xm1014", "deagle", "fiveseven", "usps", "p250", "p2000", "tec9"]
    wpsName = ["AK-47", "AUG", "AWP", "PP-Bizon", "CZ75-Auto", "Dual Berettas", "FAMAS", "G3SG1", "Galil AR", "Glock-18", "M249", "M4A1", "M4A4", "MAC-10", "MAG-7", "MP5-SD", "MP7", "MP9", "Negev", "Nova", "P90", "R8 Revolver", "Sawed-off", "SCAR-20", "SG 553", "ssg 08", "UMP-45", "XM1014", "Desert Eagle", "Five-SeveN", "USP-S", "P250", "P2000", "Tec-9"]
    grnCode = ["hegrenade", "flashbang", "smokegrenade", "incendiarygrenade", "molotovgrenade", "decoygrenade"]
    grnName = ["HE Grenade", "Flashbang", "Smoke Grenade", "Incendiary Grenade", "Molotov", "Decoy"]
    invSum = {}
    
    for n, i in enumerate(wpsCode):
        invSum["ct_weapon_"+i] = sum_weapons(ctplayers, wpsName[n])
        invSum["t_weapon_"+i] = sum_weapons(tplayers, wpsName[n])
        
    for n, i in enumerate(grnCode):
        invSum['ct_grenade_'+i] = sum_weapons(ctplayers, grnName[n])
        invSum['t_grenade_'+i] = sum_weapons(tplayers, grnName[n])
    return invSum
        
        

In [12]:
%matplotlib inline

### returns array of dicts (features), to be used with the AWPY module
def get_features(data: list, roundStart: int, roundEnd: int):
    samples = []
    for i in range(roundStart, roundEnd):
        roundData = data[i] ### get current round data
        #print("round "+ str(i))
        for frame in roundData["frames"]:
            if frame['secondsLeft'] < 0: continue
            state = generate_game_state(frame) ### generates a simplified dict of data we can use
            ct, t = state['ct'], state['t']
            ctp, tp = frame['ct']['players'], frame['t']['players']

            ### Dictionary used for more AWPY-like features
            #Newfeatures = {   
            #    "tEqVal": t['eqVal'],
            #    "ctEqVal": ct['eqVal'],
            #    "tHp": t['hp'],
            #    "ctHp": ct['hp'],
            #    "tRem": t['playersRemaining'],
            #    "ctRem": ct['playersRemaining'],
            #    "bombsite": state['global']['bombsite'],
            #    "bombPlanted": True if state['global']['bombsite'] != "" else False, 
            #    "seconds": frame["seconds"],
            #    "scoreDiff": roundData["tScore"] - roundData["ctScore"],
            #    "winner": frame["winner"],
            #}
            datasetFeatures = {
                "time_left": frame["secondsLeft"],
                "ct_score": roundData['ctScore'],
                "t_score": roundData['tScore'],
                "map": roundData['map'],
                "bomb_planted": True if state['global']['bombsite'] != "" else False, 
                "ct_health": state['ct']['hp'],
                "t_health": state['t']['hp'],
                "ct_armor": sum_attribute(ctp, "armor"),
                "t_armor": sum_attribute(tp, "armor"),
                "ct_money": sum_attribute(ctp, "cash"),
                "t_money": sum_attribute(tp, "cash"),
                "ct_helmets": sum_attribute(ctp, "hasHelmet"),
                "t_helmets": sum_attribute(tp, "hasHelmet"),
                "ct_defuse_kits": sum_attribute(ctp, "hasDefuse"),
                "ct_players_alive": t['playersRemaining'],
                "t_players_alive": ct['playersRemaining']
            }
            datasetFeatures.update(return_team_inv(ctp, tp))
            datasetFeatures["round_winner"] = frame["winner"]
            samples.append(datasetFeatures)
    return samples
    

samples = get_features(allRounds, 0, len(allRounds))

### turn list of dicts into a dataframe
samples = pd.DataFrame(samples)
samples["map"].value_counts(normalize=True)


de_mirage    0.835645
de_dust2     0.164355
Name: map, dtype: float64

In [6]:
samples.shape

(54578, 97)

In [13]:
# output converted data
samples.to_csv("custom.csv", index=False)